# Item 48. 더 이상 사용하지 않는 객체의 레퍼런스를 제거하라

## 메모리 누수의 예시

In [2]:
// 누수가 발생하는 코드다
class TestClass {
    fun test() {
        logError = {
        val message = "${this::class::simpleName}"
            throw IllegalArgumentException(message)
        }
    }

    companion object {
        var logError: (() -> Unit)? = null
    }
}


true
false


- 위의 코드 로직을 잘 따라가면 logError는 static이므로 초기화시점후터 애플리케이션의 종료시점까지 생명주기를 같이하고, 이때 람다를 담기때문에 익명 객체가 저장될것을 예상할 수 있다.
- test 함수 실행시 익명 객체가 생성될텐데 그때 익명객체안에서 "${this::class::simpleName}"으로 TestClass 인스턴스를 참조하면서 클래스의 이름을 구하고 있다.
- 따라서 애플리케이션 종료시점까지 TestClass는 계속 참조되어 메모리 해제 대상이 되지 않고 메모리 누수가 발생한다.
 

In [ ]:
// 누수가 발생하지 않는 코드다
class TestClass {
    fun test() {
        val message = "${this::class::simpleName}"
        logError = {
            throw IllegalArgumentException(message)
        }
    }

    companion object {
        var logError: (() -> Unit)? = null
    }
}


- 위처럼 메시지를 로컬스코프 변수로 저장하고 해당 변수만 익명객체가 캡쳐링하게하면 TestClass는 참조대상에서 벗어나게되고 누수가 되지 않는다.


## 람다를 사용 / 참조할경우 메모리누수가 자주 발생한다
- 캡쳐링되는걸 항상 조심하자
- 람다를 변수로 저장시엔 사용되지 않을때 적절히 해제하는걸 습관화들이자

## 메모리와 성능 vs 가독성 확장성
- 둘은 경쟁관계일때도 협력관계일때도 있음
- 경쟁관계일때는 일반적으로 가독성 확장성에 손을 들어준다
- 예외적으로 라이브러리 구현시에는 메모리와 성능을 우선시하자
- 캐싱할때 soft ref, weak ref를 고려하자
- 사실 서버사이드에서는 메모리누수를 크게 신경쓰진 않는다
- 누적되는 메모리릭으로 oom이 날 수준이여야 확인하는 수준... 아무래도 앱개발까지 염두해둔 책이다보니 신경을 많이 쓰는거같다.
- 그럼에도 무분별한 람다 사용으로 메모리릭이 날걸 조심하는건 중요

